In [1]:
from qiskit import QuantumCircuit, transpile, execute, IBMQ
from qiskit.circuit import Delay
from states import Key, Ciphertext
from scheme_parameters import SchemeParameters
from encryption_circuit import encrypt
from decryption_circuit import create_decryption_circuit, create_decryption_circuit_for_deletion
from deletion_circuit import delete
from attack_circuit import breidbart_measurement
from qiskit.providers.aer import AerSimulator
from qiskit.test.mock import FakeMontreal
from utils import random_bit_string
import shutil
from datetime import datetime
import experiment
import typing

In [2]:
# Experiment parameters
execution_datetime = datetime.now()
experiment_id = f"cmc-3-{execution_datetime}".replace(":", "-").replace(".", "_")
system_string = "ibmq_montreal" # Or insert other system name here
microsecond_delay = 5 # Delay between preparing the qubits and the first measurement, whether deletion or decryption
folder_prefix = "data"
folder_path = f"{folder_prefix}/{experiment_id}"
circuits = [] # We will save all the circuits to serialize

# Backend parameters
provider = IBMQ.load_account()

# Ideal simulator
# backend = AerSimulator()

# Simulator with noise model
# noise_model = FakeMontreal()
# backend = AerSimulator.from_backend(noise_model)

# Real backend
backend = provider.get_backend("ibmq_montreal")

# Qiskit parameters
optimization_level = 2
shots = 1000

In [3]:
def run_and_measure(circuit: QuantumCircuit, draw: bool = False) -> typing.Dict[str, int]:
    """Transpiles and runs the QuantumCircuit, and returns the resulting counts of the execution."""
    if draw:
        print(circuit.draw())
    transpiled_circuit = transpile(
        circuit, backend=backend, optimization_level=optimization_level)
    transpiled_circuit = typing.cast(QuantumCircuit, transpiled_circuit)
    if draw:
        print(transpiled_circuit.draw())
    result = backend.run(transpiled_circuit, shots=shots).result()
    counts = result.get_counts()
    # Reverse the string since the most significant qubit is at the 0th index of the measurement string
    reversed_keys_dict = {}
    for key, value in counts.items():
        reversed_keys_dict[key[::-1]] = value
    circuits.extend([circuit, transpiled_circuit])
    return reversed_keys_dict

In [4]:
# Encrypt from new states
scheme_params = SchemeParameters.generate_from_lambda(1)
key = Key.generate_key(scheme_params)
message = random_bit_string(scheme_params.n)
ciphertext = encrypt(message, key, scheme_params)
if microsecond_delay > 0:
    ciphertext.circuit.delay(microsecond_delay, range(ciphertext.circuit.num_qubits), unit="us")
original_circuit = ciphertext.circuit.copy() # We might modify this circuit in subsequent steps
circuits.append(original_circuit)

In [5]:
# # Encrypt from existing states
# old_experiment = experiment.Experiment.reconstruct_experiment_from_folder("data/cumc-delay-0")
# scheme_params = old_experiment.parameters
# key = old_experiment.key
# message = old_experiment.message
# ciphertext = old_experiment.ciphertext
# ciphertext.circuit = old_experiment.circuits[0].copy()
# ciphertext.circuit.data = [instruction for instruction in ciphertext.circuit.data if not isinstance(instruction[0], Delay)]
# if microsecond_delay > 0:
#     ciphertext.circuit.delay(microsecond_delay, range(ciphertext.circuit.num_qubits), unit="us")
# original_circuit = ciphertext.circuit.copy()
# circuits.append(original_circuit)

In [6]:
# Test 1 - honest delete
deletion_circuit_test1 = delete(ciphertext)
deletion_counts_test1 = run_and_measure(deletion_circuit_test1)

ciphertext.circuit = original_circuit.copy() # To make it easier to re-run cells

In [7]:
# Test 2 - decrypt
decryption_circuit_test2 = create_decryption_circuit_for_deletion(key, ciphertext)
decryption_counts_test2 = run_and_measure(decryption_circuit_test2)

ciphertext.circuit = original_circuit.copy() # To make it easier to re-run cells

In [8]:
# Test 3 - honest delete and then decrypt
combined_circuit_test3 = delete(ciphertext)
# A measurement is always in the computational basis, so measuring again for decryption would be redundant
combined_counts_test3 = run_and_measure(combined_circuit_test3)

ciphertext.circuit = original_circuit.copy() # To make it easier to re-run cells

In [9]:
# Test 4 - malicious delete and then decrypt
breidbart_circuit_test4 = breidbart_measurement(ciphertext)
# A measurement is always in the computational basis, so measuring again for decryption would be redundant
combined_counts_test4 = run_and_measure(breidbart_circuit_test4)

ciphertext.circuit = original_circuit.copy() # To make it easier to re-run cells

In [10]:
# Test 5 - tamper detection
decryption_circuit_test5 = create_decryption_circuit_for_deletion(key, ciphertext)
ciphertext.circuit = decryption_circuit_test5
deletion_circuit_test5 = delete(ciphertext)
raw_counts_test5 = run_and_measure(deletion_circuit_test5)

ciphertext.circuit = original_circuit.copy()

In [11]:
# Create experiment and export to a folder
exp = experiment.Experiment(
    experiment_id=experiment_id,
    execution_datetime=execution_datetime,
    execution_shots=shots,
    optimization_level=optimization_level,
    backend_system=system_string,
    microsecond_delay=microsecond_delay,
    folder_path=folder_path,
    parameters=scheme_params,
    key=key,
    ciphertext=ciphertext,
    message=message,
    deletion_counts_test1=deletion_counts_test1,
    decryption_counts_test2=decryption_counts_test2,
    combined_counts_test3=combined_counts_test3,
    combined_counts_test4=combined_counts_test4,
    raw_counts_test5=raw_counts_test5,
    circuits=circuits,
)
exp.export_to_folder()

In [12]:
# Create zip file
shutil.make_archive("data_export", "zip", folder_prefix);